In [38]:
import pandas as pd
import numpy as np
import os
import networkx as nx
import random
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from pylab import rcParams
from sklearn.model_selection import train_test_split
rcParams['figure.figsize'] = 18, 15
%matplotlib inline

In [6]:
ideologies_dir = "./"
followers_dir = "../followers/"
geolocations_dir = "./"

In [7]:
ideals1 = pd.read_csv(ideologies_dir + "user-ideal-points-201807-000000000000.csv", usecols=["id_str", "theta"])
ideals2 = pd.read_csv(ideologies_dir + "user-ideal-points-201807-000000000001.csv",  usecols=["id_str", "theta"])
ideals3 =  pd.read_csv(ideologies_dir + "user-ideal-points-201807-000000000002.csv",  usecols=["id_str", "theta"])

ideals = ideals1.append(ideals2).append(ideals3)

In [8]:
user_ids = os.listdir(followers_dir)

In [9]:
geos1 = pd.read_csv(geolocations_dir + "already_located_random_users_machine00.csv", usecols=["user_id", "longitude", "latitude", "raw_location", "country"])
geos2 = pd.read_csv(geolocations_dir + "already_located_random_users_machine01.csv", usecols=["user_id", "longitude", "latitude", "raw_location", "country"])
geos3 = pd.read_csv(geolocations_dir + "already_located_random_users_machine02.csv", usecols=["user_id", "longitude", "latitude", "raw_location", "country"])

geos = geos1.append(geos2).append(geos3)

In [10]:
sample_size = 40

In [11]:
ideal_ids = ideals["id_str"]
geo_ids = geos["user_id"]
combined_ids = np.intersect1d(np.intersect1d(ideal_ids.values, geo_ids.values), user_ids)

In [15]:
random_users = random.sample(combined_ids.tolist(), sample_size)

In [16]:
result = []
failures = 0
for i, user_id in enumerate(random_users):
    try:
        followers = pd.read_csv(followers_dir + str(random_users[i]) + "/2019__10__" + str(random_users[i]) + ".csv")["user_id_followers"].values

        for follower in followers:
            result.append({
                "node_1": user_id,
                "node_2": follower
            })
    except Exception as e:
        failures += 1

print(str(failures) + " failures (total: " + str(len(random_users)) + ")")

7 failures (total: 40)


In [17]:
h = pd.DataFrame(result)
r = np.concatenate((h["node_1"].unique(),h["node_2"].unique()), axis=0)
rl = r.tolist()
h["node_1"] = h["node_1"].apply(lambda x: rl.index(x))
h["node_2"] = h["node_2"].apply(lambda x: rl.index(x))
h.to_csv("followerified_with_geo.csv", index=False)

In [ ]:
# g = nx.from_edgelist(h.values.tolist())
# nx.draw(g, with_labels=True)

In [18]:
embedding = pd.read_csv("follower-embedding20.csv")
embedding

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,0.050595,0.014656,0.114790,0.033866,0.375090,0.018933,0.221602,0.093643,0.445510,0.052271,0.028633,0.118936,0.298346,0.683390,0.060019,0.090350
1,0.048613,0.011486,0.681034,0.096375,0.278722,0.053271,0.019319,0.033441,0.405171,0.094503,0.200696,0.204499,0.179130,0.368560,0.081113,0.112682
2,0.043216,0.022189,0.077560,0.065755,0.390839,0.033392,0.146820,0.080941,0.504383,0.026809,0.103470,0.031678,0.100874,0.682105,0.079894,0.238048
3,0.004324,0.029669,0.078620,0.048961,0.559326,0.047039,0.213952,0.064604,0.635789,0.039229,0.245799,0.029340,0.282533,0.046879,0.070116,0.267456
4,0.005793,0.017767,0.113181,0.133852,0.353384,0.029753,0.067995,0.009812,0.290453,0.017041,0.100402,0.066948,0.191380,0.793973,0.081308,0.256112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32785,0.051839,0.121395,0.455457,0.087930,0.018033,0.210959,0.268402,0.011212,0.028887,0.185831,0.573532,0.262865,0.001096,0.164022,0.435679,0.016855
32786,0.098176,0.423666,0.288777,0.230230,0.088577,0.087639,0.066492,0.020869,0.039040,0.243920,0.404497,0.351751,0.009965,0.051661,0.482465,0.264915
32787,0.138118,0.071573,0.123132,0.245036,0.101054,0.182465,0.162503,0.009773,0.045348,0.243072,0.243121,0.181852,0.002840,0.100088,0.790022,0.207629
32788,0.082452,0.535195,0.042540,0.054717,0.017649,0.120207,0.026214,0.023353,0.005832,0.147040,0.339855,0.378016,0.018850,0.074060,0.578582,0.255580


In [19]:
grouped_embedding = embedding.groupby(lambda i: h.iloc[i]["node_1"] if i < h.shape[0] else None)

In [20]:
summed_embedding = grouped_embedding.apply(lambda x: np.log(x.sum()))
summed_embedding

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,5.122692,4.744388,7.593026,6.144301,7.618947,4.858490,6.453434,5.716147,7.916379,5.472760,6.473812,6.240388,7.391859,7.939038,5.550885,6.668291
1,-3.612572,-5.047469,3.436305,-3.884455,-0.725798,-3.494958,-3.517179,-2.821212,-0.982377,-2.984939,-1.929613,-4.849043,-0.921197,-0.464783,-2.808767,-1.699116
2,3.637586,-5.717354,0.693147,-18.938961,-12.686383,-21.066735,-23.804726,-21.085366,-18.683684,-16.390214,-22.692627,-12.435510,-15.825331,-12.564117,-15.072127,-13.296193
3,1.475045,5.635894,1.251181,3.828604,-2.635210,-1.577673,-0.125638,-0.200093,-0.096424,-0.405191,-1.461422,-2.401450,1.275929,2.235130,0.892998,-1.377923
4,4.898022,6.698036,5.404080,5.913414,5.415112,6.015549,6.912321,7.534613,4.579328,6.097804,5.307882,5.826346,3.831983,3.718562,5.849284,6.526858
5,-1.728227,5.326531,-0.892777,-0.145550,-0.860441,-0.400241,3.284982,0.628006,-1.828361,-0.158107,-1.195950,3.127280,-2.090215,-3.745627,-1.069113,0.351021
6,-3.742204,1.784204,3.046266,-2.047228,-4.066369,3.170062,0.791874,-0.554498,-2.391473,0.125018,-3.506629,0.260685,-6.253515,1.241075,5.375325,-2.574163
7,0.763618,2.394320,-0.475033,0.412467,-0.578382,5.701824,1.929827,-0.154246,-1.579710,-1.042211,0.669103,-0.912389,-0.316324,-1.149525,2.416895,-1.227813
8,-1.784223,0.196585,4.148057,0.937359,0.626443,1.927455,-0.959740,-2.707541,-2.879269,2.627638,-0.030773,-0.488025,-1.176124,-1.647373,-0.241820,-0.265323
9,3.471387,2.667357,0.300907,-1.711299,-2.714486,5.188979,-1.945398,-1.143345,-2.109983,0.286104,-0.773326,-0.633635,4.377532,-2.413002,0.673824,-1.766419


In [30]:
def j(x):
    mapped_id = rl[int(x.name)]
    
    ideology = ideals[ideals["id_str"] == int(mapped_id)]
    geo = geos[geos["user_id"] == int(mapped_id)]
    
    x["mapped_id"] = mapped_id
    x['theta'] = ideology["theta"].iloc[0]
    x["longitude"] = geo["longitude"].iloc[0]
    x["latitude"] = geo["latitude"].iloc[0]
    x["raw_location"] = geo["raw_location"].iloc[0]
    x["country"] = geo["country"].iloc[0]
    return x

dataset = summed_embedding.apply(j, axis=1)

In [31]:
dataset

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_12,X_13,X_14,X_15,mapped_id,theta,longitude,latitude,raw_location,country
0,5.122692,4.744388,7.593026,6.144301,7.618947,4.858490,6.453434,5.716147,7.916379,5.472760,...,7.391859,7.939038,5.550885,6.668291,3141702094,-0.989500,23.71622,37.97945,"athens, greece",Greece
1,-3.612572,-5.047469,3.436305,-3.884455,-0.725798,-3.494958,-3.517179,-2.821212,-0.982377,-2.984939,...,-0.921197,-0.464783,-2.808767,-1.699116,26384307,-0.302694,-87.65005,41.85003,"chicago, il",United States
2,3.637586,-5.717354,0.693147,-18.938961,-12.686383,-21.066735,-23.804726,-21.085366,-18.683684,-16.390214,...,-15.825331,-12.564117,-15.072127,-13.296193,39186951,-1.625964,-71.05977,42.35843,"boston, ma",United States
3,1.475045,5.635894,1.251181,3.828604,-2.635210,-1.577673,-0.125638,-0.200093,-0.096424,-0.405191,...,1.275929,2.235130,0.892998,-1.377923,3856144932,-1.070406,77.57976,12.93670,"bengaluru south, karnataka",India
4,4.898022,6.698036,5.404080,5.913414,5.415112,6.015549,6.912321,7.534613,4.579328,6.097804,...,3.831983,3.718562,5.849284,6.526858,850947290,2.816700,-74.17181,40.91677,nj,United States
5,-1.728227,5.326531,-0.892777,-0.145550,-0.860441,-0.400241,3.284982,0.628006,-1.828361,-0.158107,...,-2.090215,-3.745627,-1.069113,0.351021,20917875,0.193533,-88.25894,39.98614,"tolono, il",United States
6,-3.742204,1.784204,3.046266,-2.047228,-4.066369,3.170062,0.791874,-0.554498,-2.391473,0.125018,...,-6.253515,1.241075,5.375325,-2.574163,185689044,-0.511253,-74.78132,10.96854,atl,Colombia
7,0.763618,2.394320,-0.475033,0.412467,-0.578382,5.701824,1.929827,-0.154246,-1.579710,-1.042211,...,-0.316324,-1.149525,2.416895,-1.227813,76090135,-1.145919,-86.29997,32.36681,"montgomery, alabama",United States
8,-1.784223,0.196585,4.148057,0.937359,0.626443,1.927455,-0.959740,-2.707541,-2.879269,2.627638,...,-1.176124,-1.647373,-0.241820,-0.265323,2422392583,2.991099,-93.20104,43.15357,"mason city, ia",United States
9,3.471387,2.667357,0.300907,-1.711299,-2.714486,5.188979,-1.945398,-1.143345,-2.109983,0.286104,...,4.377532,-2.413002,0.673824,-1.766419,191211767,-0.620926,-74.00597,40.71427,"new york, ny",United States


In [34]:
X = dataset[dataset.columns.difference(['mapped_id', 'theta', "country","raw_location", "latitude", "longitude"])]
X

,X_0,X_1,X_10,X_11,X_12,X_13,X_14,X_15,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9
0,5.122692,4.744388,6.473812,6.240388,7.391859,7.939038,5.550885,6.668291,7.593026,6.144301,7.618947,4.858490,6.453434,5.716147,7.916379,5.472760
1,-3.612572,-5.047469,-1.929613,-4.849043,-0.921197,-0.464783,-2.808767,-1.699116,3.436305,-3.884455,-0.725798,-3.494958,-3.517179,-2.821212,-0.982377,-2.984939
2,3.637586,-5.717354,-22.692627,-12.435510,-15.825331,-12.564117,-15.072127,-13.296193,0.693147,-18.938961,-12.686383,-21.066735,-23.804726,-21.085366,-18.683684,-16.390214
3,1.475045,5.635894,-1.461422,-2.401450,1.275929,2.235130,0.892998,-1.377923,1.251181,3.828604,-2.635210,-1.577673,-0.125638,-0.200093,-0.096424,-0.405191
4,4.898022,6.698036,5.307882,5.826346,3.831983,3.718562,5.849284,6.526858,5.404080,5.913414,5.415112,6.015549,6.912321,7.534613,4.579328,6.097804
5,-1.728227,5.326531,-1.195950,3.127280,-2.090215,-3.745627,-1.069113,0.351021,-0.892777,-0.145550,-0.860441,-0.400241,3.284982,0.628006,-1.828361,-0.158107
6,-3.742204,1.784204,-3.506629,0.260685,-6.253515,1.241075,5.375325,-2.574163,3.046266,-2.047228,-4.066369,3.170062,0.791874,-0.554498,-2.391473,0.125018
7,0.763618,2.394320,0.669103,-0.912389,-0.316324,-1.149525,2.416895,-1.227813,-0.475033,0.412467,-0.578382,5.701824,1.929827,-0.154246,-1.579710,-1.042211
8,-1.784223,0.196585,-0.030773,-0.488025,-1.176124,-1.647373,-0.241820,-0.265323,4.148057,0.937359,0.626443,1.927455,-0.959740,-2.707541,-2.879269,2.627638
9,3.471387,2.667357,-0.773326,-0.633635,4.377532,-2.413002,0.673824,-1.766419,0.300907,-1.711299,-2.714486,5.188979,-1.945398,-1.143345,-2.109983,0.286104


In [36]:
y = dataset[["theta", "latitude", "longitude"]]
y

,theta,latitude,longitude
0,-0.989500,37.97945,23.71622
1,-0.302694,41.85003,-87.65005
2,-1.625964,42.35843,-71.05977
3,-1.070406,12.93670,77.57976
4,2.816700,40.91677,-74.17181
5,0.193533,39.98614,-88.25894
6,-0.511253,10.96854,-74.78132
7,-1.145919,32.36681,-86.29997
8,2.991099,43.15357,-93.20104
9,-0.620926,40.71427,-74.00597


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
model.score(X_test, y_test)

In [ ]:
# http://cig.fi.upm.es/articles/2015/Borchani-2015-WDMKD.pdf